In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


In [58]:

# Configure the map plot
# YOUR CODE HERE
map_plot_cities_df = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 600,
    size="Humidity",  # Size of the points based on humidity
    scale= 0.8,
    color="Humidity",  # Color of the points based on humidity
    cmap='viridis',     # Colormap for better visualization
    hover_cols = ["Humidity"]
)

# Display the map
map_plot_cities_df

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
cities_criteria = city_data_df[(city_data_df['Cloudiness'] == 0)]
clean_cities = cities_criteria.dropna()
clean_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
32,32,cape town,-33.9258,18.4232,20.61,71,0,9.26,ZA,1666108250
44,44,collierville,35.0420,-89.6645,7.11,47,0,5.66,US,1666108151
...,...,...,...,...,...,...,...,...,...,...
542,542,bilma,18.6853,12.9164,31.90,18,0,4.17,NE,1666108548
546,546,arcata,40.8665,-124.0828,10.66,93,0,1.54,US,1666108260
550,550,pali,25.7667,73.3333,27.18,26,0,1.38,IN,1666108552
559,559,lompoc,34.6391,-120.4579,17.55,100,0,3.60,US,1666108556


In [59]:
hotel_pd = clean_cities.get(
    ["City",                             
     "Country",                             
     "Lat",                             
     "Lng",                             
     "Humidity"])
hotel_df = pd.DataFrame(hotel_pd)

In [60]:
hotel_df["Hotel Name"] = " "

In [61]:
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,farsund,NO,58.0948,6.8047,100,
7,carnarvon,AU,-24.8667,113.6333,76,
9,saint-pierre,RE,-21.3393,55.4781,68,
32,cape town,ZA,-33.9258,18.4232,71,
44,collierville,US,35.0420,-89.6645,47,


In [62]:
# Define longitude and latitude for your location
longitude = 6.8047  # Example longitude
latitude = 58.0948    # Example latitude

# Set parameters to search for a hotel
radius = 10000
limit = 20
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apikey":geoapify_key
}
# Set base URL
base_url = "https://api.geoapify.com/v2/places"


In [65]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apikey":geoapify_key
}
# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Assuming you have a 'City' column
    City = hotel_df.loc[index, "City"]
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
farsund - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
cape town - nearest hotel: No hotel found
collierville - nearest hotel: No hotel found
kapaa - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
banda - nearest hotel: No hotel found
nalut - nearest hotel: No hotel found
makakilo city - nearest hotel: No hotel found
qostanay - nearest hotel: No hotel found
esperance - nearest hotel: No hotel found
saint george - nearest hotel: No hotel found
zhangye - nearest hotel: No hotel found
cedar city - nearest hotel: No hotel found
lichuan - nearest hotel: No hotel found
half moon bay - nearest hotel: No hotel found
brenham - nearest hotel: No hotel found
puerto del rosario - nearest hotel: No hotel found
kahului - nearest hotel: No hotel found
geraldton - nearest hotel: No hotel found
avera - nearest hotel: No hotel found
challapata - nearest hotel: No hotel found
gat - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
1,farsund,NO,58.0948,6.8047,100,No hotel found
7,carnarvon,AU,-24.8667,113.6333,76,No hotel found
9,saint-pierre,RE,-21.3393,55.4781,68,No hotel found
32,cape town,ZA,-33.9258,18.4232,71,No hotel found
44,collierville,US,35.0420,-89.6645,47,No hotel found
...,...,...,...,...,...,...
542,bilma,NE,18.6853,12.9164,18,No hotel found
546,arcata,US,40.8665,-124.0828,93,No hotel found
550,pali,IN,25.7667,73.3333,26,No hotel found
559,lompoc,US,34.6391,-120.4579,100,No hotel found


In [68]:
import requests

url = "https://api.geoapify.com/v2/places"  # Replace with the actual API endpoint

headers = {"Authorization": "b8405f6c5507408696785cceb5434308"  # Replace with your API key
}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    print("API key is valid.")
else:
    print(f"API key is invalid. Status code: {response.status_code}, Message: {response.text}")

API key is invalid. Status code: 400, Message: {"statusCode":400,"error":"Bad Request","message":"\"value\" must contain at least one of [type, categories]"}
